# Backbone congelada y descongelada

## Backbone y head

Antes de nada hay que explicar qué son el backbone y el head de una red neuronal convolucional. Como ya vimos, una arquitectura de red convolucional se compone de un montón de capas convolucionales, pooling y funciones no lineales repetidas, y por último una o varias capas de tipo fully connected. Por tanto se suele llamar backbone a la parte donde está todo el conjunto de capas convolucionales, y head a la capa o capas fully connected

![CNN](Imagenes/CNN.jpeg)

En la anterior imagen, el *feature learning*, o extractor de características, corresponde al *backbone* y la parte de *classification* corresponde a la head

Esta separación de la red se hace porque el backbone es el extractor de características (feature extractor), es decir, es capaz de sacar todas las características importantes de la imagen, y luego una head es la que es capaz de realizar por ejemplo la clasififcación analizando esas características y metiéndolas en una pequeña red fully connected.

Por tanto, una misma backbone puede ser usada para distintas tareas solo cambiando la head

## Congelar el backbone

Como hemos dicho, una misma backbone puede servir para distintas tareas solo poniendo distintas heads. Pero qué pasa si no tenemos heads para una nueva tarea, tendremos que crear una nueva y entrenarla.

Sin embargo cuando entrenamos una red, la entrenamos entera, tanto el backbone como la head. Pero entrenar el backbone no tiene sentido, ya que ya sabe extraer las características importantes de las imágenes. Por lo que reentrenarlo, solo puede llevar a sobreentrenamientos, y tardar mucho más (ya que al haber más capas, hay más cálculos) en cada epoch

Puede que tengamos un problema muy parecido a Imagenet, por ejemplo, queremos una red que sepa distinguir entre perros y gatos. Las redes preentrenadas para Imagenet ya saben clasificar perros y gatos, es decir, saben sacar las características de estos en una imagen, por lo que no tiene sentido reentrenarla

Para solucionar esto, lo que se hace es congelar el backbone. Así, en el ejemplo de gatos y perros, vamos a tener una backbone que ya sabe sacar las características, pero que al entrenar la red no se va a modificar. Además le vamos a añadir una nueva head, que solo clasifica entre perros y gatos, que esta sí que va a hacer falta entrenarla

## Como congelar en Pytorch

En Pytorch, cada parámetro de la red tiene un atributo llamado `data` y otro llamado `requires_grad`

In [27]:
from torchvision.models import resnet50, ResNet50_Weights

model = resnet50(weights=ResNet50_Weights.DEFAULT)
for i, param in enumerate(model.parameters()):
    if i < 1:
        print(type(param.data), param.requires_grad)

<class 'torch.Tensor'> True


Por lo que para congelar el backbone podemos congelar toda la red, menos la última capa fully connected

In [28]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
for i, param in enumerate(model.parameters()):
    param.requires_grad = False
model.fc.requires_grad = True

Al hacer esto, Pytorch no calculará los gradientes en el backbone, solo en la capa fully connected, por lo que solo actualizará los pesos de la capa fully connected

## Ejemplo de entrenamiento con backbone congelada

Vamos a ver un ejemplo de entrenamiento de una red con el backbone congelado, para reconocer entre gatos y perros, para ello usaremos el dataset [cats vs dogs](https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset). Lo descargo desde kaggle y lo guardo en `data/cats_vs_dogs`

### EDA

Vemos el archivo que se ha descargado

In [33]:
!ls data/cats_vs_dogs

cats_vs_dogs.zip


Lo descomprimimos

In [34]:
!unzip -q data/cats_vs_dogs/cats_vs_dogs.zip -d data/cats_vs_dogs

Vemos qué tenemos ahora

In [35]:
!ls data/cats_vs_dogs

 cats_vs_dogs.zip  'MSR-LA - 3467.docx'   PetImages  'readme[1].txt'


Aprovechamos y borramos el *.zip* para ahorrar espacio

In [38]:
!rm data/cats_vs_dogs/cats_vs_dogs.zip

Vemos qué hay dentro de `PetImages`

In [39]:
!ls data/cats_vs_dogs/PetImages

Cat  Dog


Hay una carpeta con fotos de gatos y otra con fotos de perros

In [46]:
!ls data/cats_vs_dogs/PetImages/Cat head -5

ls: opción incorrecta -- «5»
Pruebe 'ls --help' para más información.


## Ejemplo de entrenamiento con backbone descongelada

## Entrenamiento en 2 pasos (cambiar nombre)